In [4]:
import os,shutil
from PIL import Image
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA

In [5]:
cur_dir = os.getcwd()

dataset_dir = os.path.join(cur_dir,"Dataset")

labels_dir = os.path.join(dataset_dir,"labels")
frames_dir = os.path.join(dataset_dir,"frames")

n_lecs = len(os.listdir(labels_dir))

listOfLectures = []

for lec in sorted(os.listdir(frames_dir)):
    frame_list = []
    lec_path = os.path.join(frames_dir,lec)
    for frame in sorted(os.listdir(lec_path)):
        frame_list.append([os.path.join(lec_path,frame),])
    listOfLectures.append(frame_list)

# print(listOfLectures[:2])

In [6]:
lec_num = 0
for csv in sorted(os.listdir(labels_dir)):
    csv_path = os.path.join(labels_dir,csv)
    with open(csv_path,'r') as input:
        data = input.read()
        data = data.split()
        for i in range(len(data)):
            listOfLectures[lec_num][i].append(int(data[i]))
    lec_num+=1

In [7]:
# listOfLectures : list of lectures where each lecture contains a list of form [frame_number, whether hit or miss ]
im = cv2.imread(listOfLectures[0][134][0],0)
# cv2.imshow('image',im)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# print(im.dtype)
x = cv2.calcHist([im],[0],None,[256],[0,256])
# print(x)
# plt.hist(im.ravel(),256,[0,256])
# plt.show()

In [8]:
def ECR(frame, next_frame, width, height, crop=True, dilate_rate = 5):
    safe_div = lambda x,y: 0 if y == 0 else x / y
    if crop:
        startY = int(height * 0.1)
        endY = int(height * 0.9)
        startX = int(width * 0.1)
        endX = int(width * 0.9)
        frame = frame[startY:endY, startX:endX]
        next_frame = next_frame[startY:endY, startX:endX]

    edge = cv2.Canny(frame, 0, 200)
    dilated = cv2.dilate(edge, np.ones((dilate_rate, dilate_rate)))
    inverted = (255 - dilated)
    edge2 = cv2.Canny(next_frame, 0, 200)
    dilated2 = cv2.dilate(edge2, np.ones((dilate_rate, dilate_rate)))
    inverted2 = (255 - dilated2)
    log_and1 = (edge2 & inverted)
    log_and2 = (edge & inverted2)
    pixels_sum_new = np.sum(edge)
    pixels_sum_old = np.sum(edge2)
    out_pixels = np.sum(log_and1)
    in_pixels = np.sum(log_and2)
    return max(safe_div(float(in_pixels),float(pixels_sum_new)), safe_div(float(out_pixels),float(pixels_sum_old)))

In [ ]:
lectureFeatureList =[]

for i in range(len(listOfLectures)):
    # i is lec number
    pca_comp = 5
    pca = PCA(n_components=pca_comp)
    sample_image = cv2.imread(listOfLectures[i][0][0],0)
    X= sample_image.ravel().reshape((1,-1))
    for j in range(1,len(listOfLectures[i])-1):
        im = cv2.imread(listOfLectures[i][j][0],0).ravel().reshape((1,-1))
        X = np.vstack((X,im))
    pca.fit(X)
    X = pca.transform(X)
    
    #### Update it manually to number of features + 1
    lectureArray = np.ones((X.shape[0],3 + 1),dtype=np.float)
    
    f_info_next = listOfLectures[i][0]
    im_n = cv2.imread(f_info_next[0],0)
    for j in range(len(listOfLectures[i])-1):

        frame_params = []
        f_info = f_info_next
        
        f_info_next = listOfLectures[i][j+1]
        # As of now , list of form [Path,missOrHit]
        im = im_n # Grayscale image of frame
        im_n = cv2.imread(f_info_next[0],0) # Grayscale image of frame
        
        sad = np.sum(np.abs(im_n-im))
        lectureArray[j][0] = sad
        
        ecr = ECR(im,im_n,im.shape[1],im.shape[0])
        lectureArray[j][1] = ecr
        
        im_h = cv2.calcHist([im],[0],None,[256],[0,256])
        im_nh = cv2.calcHist([im_n],[0],None,[256],[0,256])
        hd = np.sum(np.abs(im_h-im_nh))
        lectureArray[j][2] = hd
        
        lectureArray[j][3] = int(f_info[1])
        
#         print(f_info)
    x_normed = (x - x.min(0)) / x.ptp(0)
    lectureArray = np.hstack((X,lectureArray))
    lectureArray = (lectureArray - lectureArray.min(0)) / lectureArray.ptp(0)
    lectureFeatureList.append(lectureArray)
#     print(lectureArray[:5])
    print("lec no." +str(i))
        

lec no.0
lec no.1
lec no.2
lec no.3
lec no.4
lec no.5
lec no.6
lec no.7
lec no.8
lec no.9
lec no.10
lec no.11
lec no.12
lec no.13
lec no.14
